In [3]:
import numpy as np
from scipy.optimize import brentq
from scipy.integrate import quad

# Physical constants
HARTREE_TO_EV = 27.211386245988  # Conversion factor from Hartree to eV

def effective_potential(r, Z, l):
    """
    Effective potential in atomic units: V_eff = -Z/r + l(l+1)/(2r^2)
    """
    if r <= 0:
        return np.inf
    coulomb = -Z / r
    centrifugal = l * (l + 1) / (2 * r**2)
    return coulomb + centrifugal

def find_turning_points(E, Z, l, r_max=500):
    """
    Find classical turning points where E = V_eff(r).
    Returns r1 (inner) and r2 (outer).
    """
    if E >= 0:
        raise ValueError("Energy must be negative for bound states")
    
    def potential_diff(r):
        return effective_potential(r, Z, l) - E
    
    if l == 0:
        r1 = 1e-10  # Avoid r=0 singularity
        r2 = brentq(potential_diff, 0.001, r_max, xtol=1e-14)
        return r1, r2
    else:
        # For l > 0, find potential minimum to bracket turning points
        from scipy.optimize import minimize_scalar
        r_min_result = minimize_scalar(lambda r: effective_potential(r, Z, l), 
                    bounds=(0.01, r_max), method='bounded')
        r_min = r_min_result.x
        V_min = r_min_result.fun
        
        if E <= V_min:
            raise ValueError(f"Energy {E:.6f} is below potential minimum {V_min:.6f}")
        
        r1 = brentq(potential_diff, 0.001, r_min, xtol=1e-14)
        r2 = brentq(potential_diff, r_min, r_max, xtol=1e-14)
        return r1, r2

def compute_action_integral(E, Z, l):
    """
    Compute action integral: ∫[r1 to r2] √(2[E - V_eff(r)]) dr
    """
    try:
        r1, r2 = find_turning_points(E, Z, l)
    except Exception as e:
        return np.inf
    
    def integrand(r):
        V_eff = effective_potential(r, Z, l)
        if E <= V_eff:
            return 0.0
        momentum_squared = 2 * (E - V_eff)
        return np.sqrt(momentum_squared)
    
    integral, _ = quad(integrand, r1, r2, limit=1000, epsabs=1e-14, epsrel=1e-12)
    return integral

def find_eigenvalue(n, l, Z):
    """
    Find energy eigenvalue using WKB: action = π (n - l - 1/2)
    """
    if l >= n:
        raise ValueError(f"l={l} must be less than n={n}")
    
    target_action = np.pi * (n - l - 0.5)
    
    def action_error(E):
        if E >= 0:
            return 1e10
        try:
            computed_action = compute_action_integral(E, Z, l)
            return computed_action - target_action
        except:
            return 1e10
    
    # Initial guess based on approximate scaling
    E_guess = -Z**2 / (2 * n**2)
    E_min = E_guess * 2.0  # Lower bound (more negative)
    E_max = E_guess * 0.1  # Upper bound (less negative)
    
    try:
        E_eigenvalue = brentq(action_error, E_min, E_max, xtol=1e-15, rtol=1e-14)
        return E_eigenvalue
    except Exception as e:
        raise RuntimeError(f"Failed to find eigenvalue for n={n}, l={l}, Z={Z}: {e}")

def calculate_energy_levels(Z, n_max, l_max):
    """
    Calculate energy levels for a hydrogen-like atom with nuclear charge Z.
    Input:
        Z: Nuclear charge (int)
        n_max: Maximum principal quantum number (int)
        l_max: Maximum angular momentum quantum number (int)
    Returns:
        Dictionary with (n, l) keys and energy information
    """
    energy_levels = {}
    orbital_names = {0: 's', 1: 'p', 2: 'd', 3: 'f', 4: 'g', 5: 'h'}
    
    print(f"\nEnergy Eigenvalues for Z={Z} (WKB Approximation)")
    print("="*50)
    print("State  | n | l | Energy (Hartree) | Energy (eV)")
    print("-"*50)
    
    for n in range(1, n_max + 1):
        for l in range(min(n, l_max + 1)):
            try:
                E_hartree = find_eigenvalue(n, l, Z)
                E_eV = E_hartree * HARTREE_TO_EV
                orbital = f"{n}{orbital_names.get(l, f'l={l}')}"
                
                energy_levels[(n, l)] = {
                    'orbital': orbital,
                    'E_hartree': E_hartree,
                    'E_eV': E_eV
                }
                
                print(f"{orbital:6} | {n} | {l} | {E_hartree:14.6f} | {E_eV:10.3f}")
            except Exception as e:
                print(f"Failed for n={n}, l={l}: {e}")
    
    return energy_levels

ModuleNotFoundError: No module named 'scipy'

In [ ]:

# Example usage
if __name__ == "__main__":
    # Example: Hydrogen (Z=1)
    calculate_energy_levels(Z=1, n_max=3, l_max=2)
    
    # Example: He+ (Z=2)
    calculate_energy_levels(Z=2, n_max=3, l_max=2)